In [ ]:
#Importing libraries
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
#Preprocessing dataset
batch_size = 32
img_height = 180
img_width = 180
#Training set
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'TLD_dataset-20201130T034909Z-001/TLD_dataset/training_set',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
#Validation set
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'TLD_dataset-20201130T034909Z-001/TLD_dataset/test_set',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
class_names = train_ds.class_names
print(class_names)

In [ ]:
"""
Performance tuning

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

"""

In [ ]:
#Creating model

data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(3)
])

In [ ]:
#Compiling model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
#Training model
epochs = 10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
#Plot acc. vs loss for Training and Validation sets
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#Predicting on images(green)
PIL.Image.open('TLD_dataset-20201130T034909Z-001/TLD_dataset/single_prediction/left1116.jpg')

In [ ]:

img = keras.preprocessing.image.load_img(
    'TLD_dataset-20201130T034909Z-001/TLD_dataset/single_prediction/left1116.jpg', target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) 

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print(score)
print("\n This image is a {} signal with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
#Predicting on images(yellow)
PIL.Image.open('TLD_dataset-20201130T034909Z-001/TLD_dataset/test_set/yellow/left0447.jpg')

In [ ]:

img = keras.preprocessing.image.load_img('TLD_dataset-20201130T034909Z-001/TLD_dataset/test_set/yellow/left0447.jpg', target_size=(img_height, img_width))

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) 

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print(score)
print("\n This image is a {} signal with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
#Predicting on images(red)
PIL.Image.open('TLD_dataset-20201130T034909Z-001/TLD_dataset/single_prediction/left0721.jpg')

In [ ]:

img = keras.preprocessing.image.load_img(
    'TLD_dataset-20201130T034909Z-001/TLD_dataset/single_prediction/left0721.jpg', target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print("\n This image is a {} signal with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)